# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [3]:
#your code here
spaceship.shape

(8693, 14)

**Check for data types**

In [4]:
#your code here
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


**Check for missing values**

In [5]:
#your code here
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [6]:
#your code here
spaceship_clean = spaceship.dropna()
spaceship_clean.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [7]:
#your code here
# Extract the deck component from the Cabin column
spaceship_clean['Deck'] = spaceship_clean['Cabin'].str[0]



C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_25580\4245816600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_clean['Deck'] = spaceship_clean['Cabin'].str[0]


In [8]:
# Now that we have the Deck, drop the original Cabin column
spaceship_clean = spaceship_clean.drop(columns=['Cabin'])

# Check unique values to ensure we have the desired {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}
print(spaceship_clean['Deck'].unique())

['B' 'F' 'A' 'G' 'E' 'C' 'D' 'T']


- Drop PassengerId and Name

In [9]:
#your code here
spaceship_clean = spaceship_clean.drop(columns=['PassengerId', 'Name'])

- For non-numerical columns, do dummies.

In [10]:
#your code here
# Convert CryoSleep and VIP into numerical format
spaceship_clean['CryoSleep'] = spaceship_clean['CryoSleep'].astype(int)
spaceship_clean['VIP'] = spaceship_clean['VIP'].astype(int)


In [11]:
# Perform one-hot encoding on categorical columns
# Columns that need to be encoded: HomePlanet, Destination, Deck
spaceship_clean = pd.get_dummies(spaceship_clean, columns=['HomePlanet', 'Destination', 'Deck'], drop_first=True)

In [12]:
spaceship_clean

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0,1,1,0,0,0,0,0,0
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,0,0,0,1,0,0,0,0,1,0,0
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,1,0,0,1,0,0,0,0,0,0,0
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,1,0,0,1,0,0,0,0,0,0,0
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,1,0,0,0,0,0,0,0,0,0,0
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,0,0,1,0,0,0,0,0,0,1,0
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,0,0,0,1,0,0,0,0,0,1,0
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,1,0,0,0,0,0,0,1,0,0,0


**Perform Train Test Split**

In [13]:
#your code here
x = spaceship_clean.drop(columns=['Transported'])
y = spaceship_clean['Transported']

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [15]:
#your code here
knn = KNeighborsClassifier()

- Evaluate your model's performance. Comment it

In [16]:
#your code here
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [17]:
accuracy = knn.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.4f}")

Model accuracy: 0.7905


probamos con n_neighbors=9

In [18]:
knn_9 = KNeighborsClassifier(n_neighbors=9,)

In [19]:
knn_9.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=9)

In [20]:
accuracy_knn_9 = knn_9.score(X_test, y_test)
print(f"Model accuracy: {accuracy_knn_9:.4f}")

Model accuracy: 0.7995


probamos con datos normalizados y con los datos escalados

In [21]:
# Normalize the data

normalizer = MinMaxScaler()

In [22]:
X_train_norm = normalizer.fit_transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [23]:
scaler = StandardScaler()

In [24]:
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

Probamos los modelos con los datos normalizados

In [25]:
#your code here
knn_norm = KNeighborsClassifier()

In [26]:
#your code here
knn_norm.fit(X_train_norm, y_train)

KNeighborsClassifier()

In [27]:
accuracy_knn_norm = knn_norm.score(X_test_norm, y_test)
print(f"Model accuracy: {accuracy_knn_norm:.4f}")

Model accuracy: 0.7648


probamos con n_neighbors=9

In [28]:
knn_9_norm = KNeighborsClassifier(n_neighbors=9,)

In [29]:
knn_9_norm.fit(X_train_norm, y_train)

KNeighborsClassifier(n_neighbors=9)

In [30]:
accuracy_knn_9_norm = knn_9_norm.score(X_test_norm, y_test)
print(f"Model accuracy: {accuracy_knn_9_norm:.4f}")

Model accuracy: 0.7746


Probamos los modelos con los datos escalados

In [31]:
#your code here
knn_scaled = KNeighborsClassifier()

In [32]:
#your code here
knn_scaled.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [33]:
accuracy_knn_scaled = knn_scaled.score(X_test_scaled, y_test)
print(f"Model accuracy: {accuracy_knn_scaled:.4f}")

Model accuracy: 0.7874


probamos con n_neighbors=9

In [34]:
knn_9_scaled = KNeighborsClassifier(n_neighbors=9,)

In [35]:
knn_9_scaled.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=9)

In [37]:
accuracy_knn_9_scaled = knn_9_scaled.score(X_test_scaled, y_test)
print(f"Model accuracy: {accuracy_knn_9_scaled:.4f}")

Model accuracy: 0.7943
